# Spammer Detection



## Introduction


### Motivation

With the popularization of the computer and the developing of the internet, ranking websites such as Yelp, Imdb, IGN become popular in the society. More and more people rely on review system when making their options. In order to build up a robust review system, it is very important to find out which reviews are reliable and fair. 
This project is going to find out what kind of features in either reviews or users might end up with a spam or spammer. We choose Yelp restaurant reviews in Pittsburgh area as our datasets. Apply different machine learning methods to get the hypothesis features that might led to a spam or spammer.

### Abstract

In order to get all the reviews and relevant features from Yelp in a single restaurant, we use python beautifulsoup4 package to acquire these data. First, we focus on every single reviews and figure out what kinds of features is most likely to be a spam feature. However, even if we can sort out the spam precisely and delete it, these spams might come out again and again in our review system. Therefore, Next step, we focus on finding the specific features in users that a spammers may have. We can track down those users and delete their membership. The final step is to find out the possible restaurant that is hiring a spammer. We can warn those restaurant or cancel their post if need it. With all the features we got in our models, we can apply these to other review system and make a more robust review system without spams and spammers.   

### Required Libraries

In [3]:
import nltk
import nltk.tokenize.punkt
from nltk.tokenize import word_tokenize
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import wordnet as wn
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.classify import NaiveBayesClassifier
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize
from nltk.sentiment.util import *

import io, time, json
import sys

import requests
from bs4 import BeautifulSoup
from yelpAPI import *

from collections import OrderedDict
import codecs
import re

import random
from time import sleep

# import yelp client library
from yelp.client import Client
from yelp.oauth1_authenticator import Oauth1Authenticator

C:\Users\Oscar Wang\Anaconda3\envs\PDS\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


ImportError: No module named requests

## Data acqusition

### Feature Collection

Basically, we are extensively collecting everything from Yelp.com. Since we are not only interested in review text but also meta data of users and restaurants, we can categorize our data into the following three categories.

Some of the features can be extracted directly from Yelp pages but others have to be aggregated or even calculated using graph relations between restaurants and users. 
* Review level:
    - Review time
    - Review length
    - Review rating
    - Percentage of capital words
    - Subjectivity/Objectivity of review
    - Polarity of review  
    - Language model (unigram, bigram)
* User level:
    - Number of reviews
    - Number of friends
    - Number of photos
    - Has profile photo or not
    - Yelp Since
    - Rating Distribution
    - Average Rating
    - Ratio of Positive Rating
    - Ratio of Negative Rating
    - Rating Deviation 
    - Active duration
    - Entropy of review ratings
    - Average Review Length
    - Average Percentage of capital words
    - Average Subjectivity/Objectivity of review
    - Average Polarity of review  
* Restaurant level: 
    - Number of reviews
    - Number of recommended reviews
    - Number of non recommended reviews
    - Average Rating
    - Ratio of Positive Rating
    - Ratio of Negative Rating
    - Max review in a day
    - Entropy of review ratings
    - Average Review Length
    - Average Percentage of capital words
    - Average Subjectivity/Objectivity of review
    - Average Polarity of review 




### Crawler

We starts with 1000 restaurants in Pittsburgh and obtained their reviews and a list of users who reviewed this restaurant. These gave us about 36000 users and 86000 reviews. We then use another crawler to crawl user level data. One problem we are facing is that when we were collecting user level data, we found that yelp only provide review information about recommend or not recommend for restaurants, which in our case, is the label on restaurant pages. (On user pages, yelp shows all reviews with indicating whether they are recommended or not.)
To overcome this problem and get correct information at user level, we have to crawl one more level deeper. We get another restaurant list from those 36000 users and we crawl every restaurant they visited and get the correct label of those reviews we encountered in user pages.
	

In [ ]:
# Restaurant review crawler:
# parse recommended page:

def parse_page(html):

	soup = BeautifulSoup(html, 'html.parser')

	l = list()

	try:
		recommend_review_container = soup.find('div', class_="review-list")
		reviews = recommend_review_container.find_all('div', class_="review review--with-sidebar")

		for r in reviews:
			try:
				review_id = r.get('data-review-id')

				user_photo = r.find('a', class_="user-photo js-analytics-click")
				user_id = r.get('data-signup-object').split(":")[1]
				rating = r.find('div', class_="rating-very-large").get('title').split(" ")[0]
				date = re.sub(r"\s+", '', r.find('span', class_="rating-qualifier").get_text())
				text = r.find('div', class_="review-content").find('p').get_text()

				l.append({
					'review_id': review_id,
					'user_id': user_id,
					'rating': float(rating),
					'date': date,
					'text': text,		   
					})
			except:
				print ">>> EXCEPTION: a review can not be extract. Ignore"
				traceback.print_exc()
				pass
	except:
		print ">>> EXCEPTION: reviews can not be extract. Ignore"
		traceback.print_exc()
		pass


	# find next page
	next = soup.find_all('a', class_="u-decoration-none next pagination-links_anchor")

	if len(next) != 0:
		nextLink = next[0].get('href')
	else:
		nextLink = ""

	return (l, nextLink)

def extract_reviews(url):
	
	pl = list()
	pUrl = url

	while pUrl != "":
		# sleep(random.random() * 2)

		html = retrieve_html(pUrl)[1]
		pRes = parse_page(html)

		pl.extend(pRes[0])
		pUrl = pRes[1]

	return pl


In [2]:
# Restaurant review crawler:
# parse recommended page:

def parse_page_not_recommend(html):

	soup = BeautifulSoup(html, 'html.parser')

	# find reviews
	l = list()

	try:
		not_recommended_reviews = soup.find('div', class_='not-recommended-reviews')
		reviews = not_recommended_reviews.find_all('div', class_="review review--with-sidebar")

		for r in reviews:
			try:
				review_id = r.get('data-review-id')
				user_id = r.get('data-signup-object').split(":")[1]
				rating = r.find('div', class_="rating-very-large").get('title').split(" ")[0]
				date = re.sub(r"\s+", '', r.find('span', class_="rating-qualifier").get_text())
				text = r.find('div', class_="review-content").find('p').get_text()

				l.append({
					'review_id': review_id,
					'user_id': user_id,
					'rating': float(rating),
					'date': date,
					'text': text,		   
					})
			except:
				print ">>> EXCEPTION: a review can not be extract. Ignore"
				pass
	except:
		print ">>> EXCEPTION: reviews can not be extract. Ignore"
		traceback.print_exc()
		pass

	# find next page
	pagination = soup.find('div', 'pagination-links arrange_unit')
	nextLink = ""
	if pagination is not None:
		next = pagination.find_all('a', class_="u-decoration-none next pagination-links_anchor")
		if len(next) != 0:
			nextLink = next[0].get('href')

	return (l, nextLink)

def extract_unrecommend_reviews(url):
	nl = list()
	nUrl = url

	while nUrl != "":
		# sleep(random.random() * 2)

		html = retrieve_html('https://www.yelp.com' + nUrl)[1]
		nRes = parse_page_not_recommend(html)

		nl.extend(nRes[0])
		nUrl = nRes[1]

	return nl

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-2-38f465492763>, line 31)

In [ ]:
# main function to load reviews of 1000 restaurants in Pittsburgh

f_reviews_content = io.open('outputData/reviews_content_' + subset + '.tsv', 'w', encoding='utf8')
f_userIdMapping = io.open('outputData/user_id_mapping_' + subset + '.tsv', 'w', encoding='utf8')
f_metaData = io.open('outputData/metaData_' + subset + '.tsv', 'w', encoding='utf8')

userIdDict = {}

# read 1000 restaurants in Pittsburgh

f = open('outputData/businessesIdMapping.tsv', 'r')
businessLines = [line.rstrip('\n\r') for line in f]

cnt = 1

for businessLine in businessLines[int(subset): int(subset) + 100]:

	businessId = businessLine.split('\t')[0]

	print ">>> " + businessId + " extracting recommended reviews from: " + businessLine.split('\t')[1]
	url = 'https://www.yelp.com/biz/' + businessLine.split('\t')[1]
	reviews = extract_reviews(url)
	print ">>> extrated " + str(len(reviews)) + " from " + businessLine.split('\t')[1]


	for review in reviews:

		# added user to user set
		if review['user_id'] not in userIdDict:
			userIdDict[review['user_id']] = {'pos': 0, 'neg': 1}
		else:
			userIdDict[review['user_id']]['neg'] += 1

		f_reviews_content.write(str(cnt) + "\t" + review['review_id'] + "\t" + review['text'] + "\n")
		f_metaData.write(str(cnt) + "\t" + review['user_id'] + "\t" + businessId + "\t" + str(review['rating']) + "\t" + "1\t" + review['date'] + "\n")
		cnt += 1


	print ">>> " + businessId + "  extracting non-recommended reviews from: " + businessLine.split('\t')[1]
	url = '/not_recommended_reviews/' + businessLine.split('\t')[1]
	reviews = extract_unrecommend_reviews(url)
	print ">>> extrated " + str(len(reviews)) + " from " + businessLine.split('\t')[1]

	for review in reviews:

		# added user to user set
		if review['user_id'] not in userIdDict:
			userIdDict[review['user_id']] = {'pos': 1, 'neg': 0}
		else:
			userIdDict[review['user_id']]['pos'] += 1

		f_reviews_content.write(str(cnt) + "\t" + review['review_id'] + "\t" + review['text'] + "\n")
		f_metaData.write(str(cnt) + "\t" + review['user_id'] + "\t" + businessId + "\t" + str(review['rating']) + "\t" + "-1\t" + review['date'] + "\n")

		cnt += 1

print "... finished crawling, extracted " + str(cnt-1) + " reviews in total"

cnt = 1
for userId, count in userIdDict.iteritems():
	f_userIdMapping.write(str(cnt) + "\t" + userId + "\t" + str(count['neg']) + "\t" + str(count['pos']) + "\n")
	cnt+=1


In [3]:
# user page crawler

def parseUserReviews(html):
	soup = BeautifulSoup(html, 'html.parser')

	# find reviews
	l = list()

	try:
		# not_recommended_reviews = soup.find('div', class_='not-recommended-reviews')
		# businesses = soup.find_all('a', class_="biz-name js-analytics-click")

		reviewContainer = soup.find('div', class_='user-details_reviews')
		reviews = reviewContainer.find_all('div', class_="review")

		for r in reviews:
			try:
				reviewId = r.get('data-review-id')
				bizId = r.find('a', class_="biz-name js-analytics-click").get('href').split("/")[2]
				reviewContent = r.find('div', class_="review-content").find('p').get_text()

				l.append({
					'reviewId': reviewId,
					'bizId': bizId,
					'reviewContent': reviewContent,		   
					})
			except:
				print ">>> EXCEPTION: a review can not be extract. Ignore"
				pass

		# find next page
		pagination = soup.find('div', 'pagination-links arrange_unit')
		nextLink = ""
		if pagination is not None:
			next = pagination.find_all('a', class_="u-decoration-none next pagination-links_anchor")
			if len(next) != 0:
				nextLink = next[0].get('href')

		return (l, nextLink)

	except:
		print ">>> EXCEPTION: reviews can not be extract. Ignore"
		traceback.print_exc()
		return (l, "")
		pass


def getReviewFromUserPage(userId):

	reviewList = list()
	url = 'https://www.yelp.com/user_details_reviews_self?userid=' + userId + '&rec_pagestart=0'

	while url != "":
		html = retrieve_html(url)[1]
		res = parseUserReviews(html)

		reviewList.extend(res[0])
		url = res[1]

	return reviewList


In [ ]:
# crawl user data
# return rows of (userId, neg, pos)
def getSuspiciousUsers(inputFile, outputFile):
	print ">>> in getSuspiciousUsers"

	f_in = open('outputData_Final/' + inputFile + '.tsv', 'r')
	userParts = [line.rstrip('\n\r').split("\t") for line in f_in]

	f_out = open('outputData_Final/' + outputFile + '.tsv', 'w')
	for userPart in userParts:
		if int(userPart[2]) + int(userPart[3]) > 1 and int(userPart[3]) > 0:
			f_out.write("\t".join(userPart) + "\n")

# return rows of (reviewId, userId, businessId)
def crawlUserReviews(inputFile, outputFile1, outputFile2, start):
	print ">>> in crawlUserReviews"

	f_in = open('outputData_Final/' + inputFile + '.tsv', 'r')
	userParts = [line.rstrip('\n\r').split("\t") for line in f_in]

	print ">>> preparing to read " + str(len(userParts)) + " user reviews"

	# sleep for 2 minute between every 100 users
	for offsetIndex in range(start, len(userParts) / 100 + 1):
		offset = offsetIndex * 100
		# print offsetIndex
		f_out_meta = io.open('outputData_Final/suspiciousReviewerReviews/' + outputFile1 + "_" + str(offset) + '.tsv', 'w', encoding='utf8')
		f_out_content = io.open('outputData_Final/suspiciousReviewerReviews/' + outputFile2 + "_" + str(offset) + '.tsv', 'w', encoding='utf8')

		cnt = 1
		for part in userParts[offset: offset + 100]:
			reviews = getReviewFromUserPage(part[1])
			print ">>> " + part[0] + "  extracted " + str(len(reviews)) + " from user: " + part[1]

			for review in reviews:

				f_out_meta.write(str(cnt) + "\t" + review['reviewId'] + "\t" + part[1] + "\t" + review['bizId'] + "\n")
				f_out_content.write(str(cnt) + "\t" + review['reviewId'] + "\t" + review['reviewContent'] + "\n")

				cnt += 1

		f_out_meta.close()
		f_out_content.close()

		print ">>> file: " + str(offset) + " is written. sleep for 2 minutes..."
		sleep(120)

In [5]:
# helper functions
def getBizFromUserPage(userId):

	bizList = list()
	url = 'https://www.yelp.com/user_details_reviews_self?userid=' + userId + '&rec_pagestart=0'

	while url != "":
		html = retrieve_html(url)[1]
		res = parseUserPage(html)

		bizList.extend(res[0])
		url = res[1]

	return bizList


def getMetaDataFromBiz(bizId):

	url = "https://www.yelp.com/biz/" + bizId
	html = retrieve_html(url)[1]
	soup = BeautifulSoup(html, 'html.parser')
	infoContainer = soup.find('div', class_='biz-main-info embossed-text-white')
	stars = infoContainer.find('div', class_='rating-very-large').get('title').split(" ")[0]
	recommendCountText = infoContainer.find('span', 'review-count rating-qualifier').get_text()
	recommendCount = ' '.join(recommendCountText.split()).split(" ")[0]

	try:
		notRecommendedContainer = soup.find('div', class_= 'not-recommended ysection')
		notRecommendCountText = notRecommendedContainer.find('a', class_='subtle-text').get_text()
		notRecommendCount = ' '.join(notRecommendCountText.split()).split(" ")[0]

	except:
		print ">>> EXCEPTION: could not find notRecommend Count. return 0"
		notRecommendCount = '0'

	url = "https://www.yelp.com/not_recommended_reviews/" + bizId

	return [stars, recommendCount, notRecommendCount]

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-5-a24ef95af51a>, line 33)

## Modeling Spam 

Normally, spam review usually has some special features that we can use for detecting. As mentioned above, our ultimate goal is to determine whether restaurants have people writing spam review for them or not. Hence, the first step is to determine if a single review content is fake. In order to do that, we first use text classification that we learned in class to do simple spam modeling.


### Getting the Labels

Before learning the classifier, we need the true labels of our training review data. Here we use the labels provided by Yelp website, which indicates that a review is “recommended” or “not recommended”.

### Text Classification

To build our text classifier, we process the review text, use bag of words and tfidf techniques to the training data. As mentioned in the lecture slide, bag of words work pretty effectively but throwing information. So we also use N-gram to improve our classification.

### Text Features

### Subjectivity and Objectivity

In [ ]:
f = open('inputData/metaData_all.tsv', 'r')
metaDataLines = [line.rstrip('\n\r') for line in f]

f = codecs.open('inputData/reviews_content_all.tsv', 'r', 'utf8')
reviewsContentLines = [line.rstrip('\n\r') for line in f]

n_instances = 100
subj_docs = [(sent, 'subj') for sent in subjectivity.sents(categories='subj')[:n_instances]]
obj_docs = [(sent, 'obj') for sent in subjectivity.sents(categories='obj')[:n_instances]]


train_subj_docs = subj_docs[:80]
train_obj_docs = obj_docs[:80]
training_docs = train_subj_docs+train_obj_docs
sentim_analyzer = SentimentAnalyzer()
all_words_neg = sentim_analyzer.all_words([mark_negation(doc) for doc in training_docs])

unigram_feats = sentim_analyzer.unigram_word_feats(all_words_neg, min_freq=4)
sentim_analyzer.add_feat_extractor(extract_unigram_feats, unigrams=unigram_feats)

training_set = sentim_analyzer.apply_features(training_docs)

testing_docs = reviewsContentLines[0: 50]

test_set = sentim_analyzer.apply_features(testing_docs)

trainer = NaiveBayesClassifier.train
classifier = sentim_analyzer.train(trainer, training_set)

for key,value in sorted(sentim_analyzer.evaluate(test_set).items()):
    print('{0}: {1}'.format(key, value))

### Polarity

In [ ]:
f = codecs.open('inputData/reviews_content_all.tsv', 'r', 'utf8')
reviewsContent = [line.rstrip('\n\r').split("\t")[2] for line in f]
f = open('inputData/metaData_all.tsv', 'r')
metaDataLines = [line.rstrip('\n\r') for line in f]

sentences = reviewsContent[0:1000]
categories = {
	'1.0': {'cnt': 0, 'neg': 0, 'neu': 0, 'pos': 0},
	'2.0': {'cnt': 0, 'neg': 0, 'neu': 0, 'pos': 0},
	'3.0': {'cnt': 0, 'neg': 0, 'neu': 0, 'pos': 0},
	'4.0': {'cnt': 0, 'neg': 0, 'neu': 0, 'pos': 0},
	'5.0': {'cnt': 0, 'neg': 0, 'neu': 0, 'pos': 0}
}

sid = SentimentIntensityAnalyzer()
for i in range(len(sentences)):
    print(sentences[i][0:50] + " ...")
    ss = sid.polarity_scores(sentences[i])
    print ss
    ratings = metaDataLines[i].split("\t")[3]
    categories[ratings]['cnt'] += 1
    categories[ratings]['neg'] += ss['neg']
    categories[ratings]['neu'] += ss['neu']
    categories[ratings]['pos'] += ss['pos']

for k, v in categories.iteritems():
	print ">>> ratings: " + k
	print "neg: " + str(float(v['neg'])/v['cnt'])
	print "neu: " + str(float(v['neu'])/v['cnt'])
	print "pos: " + str(float(v['pos'])/v['cnt'])

## Spammer Modeling

### Review feature model

In the review datasets, we have the label of whether it is a recommend review or not. We use that label as our output and implement 5 machine learning model to train.

- SVM (Support vector machine)
- MLP (Multilayer perceptron)
- KNN (K-nearest neighbors algorithm)
- Decision Tree
- Logistic Regression

We split all the datasets into train(80%) and validation(20%), drop the redundant column and train our models.

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [6]:
df = pd.read_csv('review_features_labeled.TSV', delimiter='\t')
train, test = train_test_split(df, test_size = 0.2, stratify = df['label'])
X_tr = train.drop(['k', 'biz', 'user', 'date', 'label'], axis=1)
y_tr = train['label']
X_te = test.drop(['k', 'biz', 'user', 'date', 'label'], axis=1)
y_te = test['label']
print float(df[df.label == 1].shape[0]) / float(df.shape[0])

0.855908367344


In [7]:
#MLP
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier()
clf.fit(X_tr,y_tr)
y_pre = clf.predict(X_te)
print np.mean(y_pre==y_te)

0.856022978876


In [8]:
#KNN
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(3)
clf.fit(X_tr,y_tr)
y_pre = clf.predict(X_te)
print np.mean(y_pre==y_te)

0.821614505416


In [9]:
#DT
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(max_depth=5)
clf.fit(X_tr,y_tr)
y_pre = clf.predict(X_te)
print np.mean(y_pre==y_te)

0.856920591227


In [10]:
#log
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(X_tr,y_tr)
y_pre = clf.predict(X_te)
print np.mean(y_pre==y_te)

0.855903297229


In [11]:
from sklearn.metrics import classification_report
target_names = ['Spammer', 'Not spammer']
print classification_report(y_te, y_pre, target_names=target_names)

             precision    recall  f1-score   support

    Spammer       0.50      0.00      0.01      2408
Not spammer       0.86      1.00      0.92     14303

avg / total       0.81      0.86      0.79     16711



## Dectecting Suspicious Restaurants

Now we are able to detect spam reviews and also capture spammers. However, from Yelp’s point of view, even if they can capture spammers and block their account, they still can keep posting spam review with new account. Hence, in order to avoid it, we might want to detect suspicious restaurants and deliver warning to them.
Since we are able to detect spams and spammers, we can track the distribution of spam review of a restaurant over time. Here we first make some assumptions. For example, we might assume that if a restaurant has relatively more spam review in early stage, it might be hiring people writing spam review to increase popularity. On the other hand, if a restaurant has unusual frequent spam review, it might be set up by its competitors. 

Since we are able to detect spams and spammers, we can track the distribution of spam review of a restaurant over time. Here we first make some assumptions. For example, we might assume that if a restaurant has relatively more spam review in early stage, it might be hiring people writing spam review to increase popularity. On the other hand, if a restaurant has unusual frequent spam review, it might be set up by its competitors. 

## Applying Models to other Domains

TBD



## Summary
TBD